In [5]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=4,5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: CUDA_VISIBLE_DEVICES=4,5


In [6]:
import socket
node_name = socket.gethostname()
print(node_name)

flair-node-08


In [7]:
import torch

# Get the current GPU device id
gpu_id = torch.cuda.current_device()
print("GPU Number:", gpu_id)

# Optionally, get the name of the GPU
gpu_name = torch.cuda.get_device_name(gpu_id)
print("GPU Name:", gpu_name)

GPU Number: 0
GPU Name: NVIDIA L40S


In [9]:
import os

cuda_visible_devices = os.environ.get("CUDA_VISIBLE_DEVICES")
print("CUDA_VISIBLE_DEVICES:", cuda_visible_devices)
cuda_visible_devices

CUDA_VISIBLE_DEVICES: 4,5


'4,5'